In [2]:
import numpy as np
import torch
import pandas as pd
from tqdm import tqdm
from transformers import BertModel, BertTokenizer, BertForMaskedLM, pipeline
from sentence_transformers import SentenceTransformer, util
from transformers import logging
logging.set_verbosity_error()

print(torch.cuda.is_available())
print(torch.__version__)

import os
from pathlib import Path
os.chdir(Path(os.environ["ALTHOME"]) / "Master-thesis")
# print(os.environ['ALTHOME'])
# print(os.environ['PIP_CACHE_DIR'])
# print(os.environ['TRANSFORMERS_CACHE'])

True
1.13.0+cu117


In [3]:
l = ["Ä","ä","Ö","ö","Ü","ü","ß"]
replacements = {key: str(bytes(key, encoding="utf-8"), encoding="latin-1") for key in l}

def replace(s):
    for k,v in replacements.items():
        s = s.replace(v, k)
    return s

if False:
    with open("data/SimpleGerman/spacy_easy.txt") as fp:
        text = fp.read()
    text = replace(text)
    with open("data/SimpleGerman/fixed_easy.txt", "w") as fp:
        fp.write(text)
    with open("data/SimpleGerman/spacy_normal.txt") as fp:
        text = fp.read()
    text = replace(text)
    with open("data/SimpleGerman/fixed_normal.txt", "w") as fp:
        fp.write(text)

In [4]:
model_name = "dbmdz/bert-base-german-cased"
model_name = "deepset/gbert-base"
pipe  = pipeline(model=model_name, tokenizer=model_name, task="feature-extraction")

In [5]:
from transformers import BertModel, BertTokenizer
sentence = "Dies ist eine Satz"
model_name = "deepset/gbert-base"
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)
tokens = tokenizer.encode(sentence, return_tensors="pt")
output = model.forward(tokens, output_hidden_states=True)
output_pipe = pipe(sentence)

a = output.hidden_states[-1].detach().numpy() # same as pipeline feature extraction
c = output.last_hidden_state.detach().numpy()
b = np.array(output_pipe)

print((a == b).all())
print((a == c).all())
print(f"{type(a)} - {a.shape}")
print(f"{type(b)} - {b.shape}")
print(f"{type(c)} - {c.shape}")
print(b)


True
True
<class 'numpy.ndarray'> - (1, 6, 768)
<class 'numpy.ndarray'> - (1, 6, 768)
<class 'numpy.ndarray'> - (1, 6, 768)
[[[-7.19576180e-01 -5.80246687e-01  5.33426665e-02 ... -1.22093451e+00
   -2.47317970e-01  1.05262408e-03]
  [-3.05296212e-01 -1.76625597e+00 -8.15026835e-02 ... -1.57253847e-01
    3.35350335e-02  2.63795793e-01]
  [ 2.71064460e-01  3.99326533e-01  5.00104278e-02 ... -6.03571236e-01
    2.70808160e-01 -9.88669991e-01]
  [-1.99700069e+00  8.68344784e-01 -3.31405461e-01 ... -1.72403836e+00
    9.63193104e-02 -1.04005468e+00]
  [-1.63929546e+00 -4.86749291e-01  1.62597030e-01 ... -8.14551473e-01
    5.09356298e-02 -9.36067760e-01]
  [-1.37479246e-01  6.05183095e-02 -5.09228706e-01 ...  6.99507177e-01
    5.57994843e-01 -2.54207134e-01]]]
